### TRUST_MCNet

In [5]:
# CONFIG CELL
from omegaconf import OmegaConf
from pydantic import BaseModel

class TrainingConfig(BaseModel):
    num_rounds: int = 5
    batch_size: int = 64
    learning_rate: float = 0.001
    num_clients: int = 3

config = TrainingConfig()
OmegaConf.create(config.dict())


{'num_rounds': 5, 'batch_size': 64, 'learning_rate': 0.001, 'num_clients': 3}

In [6]:
# DATASET CELL
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset
import torch

class ToNIoTDataset:
    def __init__(self, partition_id, total_partitions, config):
        np.random.seed(42 + partition_id)

        n_samples = 6000
        n_features = 42

        # Generate normal (label 0) and anomaly (label 1)
        normal = np.random.normal(0, 1, (int(0.8*n_samples), n_features))
        anomaly = np.random.normal(2, 1.5, (int(0.2*n_samples), n_features))
        X = np.vstack((normal, anomaly))
        y = np.hstack((np.zeros(normal.shape[0]), np.ones(anomaly.shape[0])))

        X, y = X[partition_id::total_partitions], y[partition_id::total_partitions]

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        self.train_loader = DataLoader(TensorDataset(torch.tensor(X_train).float(), torch.tensor(y_train).long()), batch_size=config.batch_size)
        self.test_loader = DataLoader(TensorDataset(torch.tensor(X_test).float(), torch.tensor(y_test).long()), batch_size=config.batch_size)

        self.input_dim = X.shape[1]
        self.num_classes = 2


In [8]:
# MODEL CELL
import torch.nn as nn
import torch.nn.functional as F

class SimpleModel(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, 64)
        self.out = nn.Linear(64, num_classes)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.out(x)


In [9]:
# CLIENT CELL
from flwr.client import NumPyClient

def get_weights(model):
    return [val.cpu().numpy() for _, val in model.state_dict().items()]

def set_weights(model, weights):
    params_dict = zip(model.state_dict().keys(), weights)
    state_dict = {k: torch.tensor(v) for k, v in params_dict}
    model.load_state_dict(state_dict, strict=True)

class TrustMCClient(NumPyClient):
    def __init__(self, model, train_loader, test_loader):
        self.model = model
        self.train_loader = train_loader
        self.test_loader = test_loader
        self.loss_fn = nn.CrossEntropyLoss()
        self.optim = torch.optim.Adam(self.model.parameters(), lr=config.learning_rate)

    def get_parameters(self, config):
        return get_weights(self.model)

    def fit(self, parameters, config):
        set_weights(self.model, parameters)
        self.model.train()
        for X, y in self.train_loader:
            self.optim.zero_grad()
            loss = self.loss_fn(self.model(X), y)
            loss.backward()
            self.optim.step()
        return get_weights(self.model), len(self.train_loader.dataset), {}

    def evaluate(self, parameters, config):
        set_weights(self.model, parameters)
        self.model.eval()
        correct, total = 0, 0
        loss_total = 0.0
        with torch.no_grad():
            for X, y in self.test_loader:
                preds = self.model(X)
                loss = self.loss_fn(preds, y)
                loss_total += loss.item()
                correct += (preds.argmax(1) == y).sum().item()
                total += y.size(0)
        return loss_total / total, total, {"accuracy": correct / total}


In [10]:
# STRATEGY CELL
import warnings
import numpy as np
from math import sqrt
from scipy.stats import spearmanr
from flwr.server.strategy import FedAvg
from flwr.common import parameters_to_ndarrays, ndarrays_to_parameters

warnings.filterwarnings("ignore")

class TrustMCStrategy(FedAvg):
    def __init__(
        self,
        *,
        percentile: float = 40,
        eta0: float = 0.10,
        lam: float = 0.2,
        kappa: float = 5.0,
        temp0: float = 1.0,
        temp_min: float = 0.3,
        temp_decay: float = 0.1,
        **kwargs
    ):
        super().__init__(**kwargs)
        # --- Bayesian-mirror state ---
        self.theta = np.array([1/3, 1/3, 1/3])  # α,β,γ on the 2-simplex
        self.t = 0                              # round counter
        self.lam = lam
        self.kappa = kappa
        self.eta0 = eta0
        # --- softmax params & thresholding ---
        self.percentile = percentile
        self.temp0 = temp0
        self.temp_min = temp_min
        self.temp_decay = temp_decay
        # track previous global accuracy for ΔAcc
        self.prev_global_accuracy = 0.0

    def aggregate_fit(self, server_round, results, failures):
        """Per-round:
           1) Compute Spearman ρ for each metric
           2) Bayesian-mirror update of θ
           3) Compute per-client trust scores T
           4) Dynamic percentile threshold → trusted set
           5) Softmax weighting → w_i
           6) Weighted avg of client params → new global params
        """
        if not results:
            return super().aggregate_fit(server_round, results, failures)

        # 1) Extract per-client metrics and ΔAcc
        metric_names = ["cos", "ent", "rep"]
        # build (n_clients, 3) array M and (n,) ΔAcc
        Ms = []
        ΔAcc = []
        for cid, fit_res in results:
            m = [fit_res.metrics.get(name, 0.0) for name in metric_names]
            Ms.append(m)
            acc = fit_res.metrics.get("accuracy", 0.0)
            ΔAcc.append(acc - self.prev_global_accuracy)
        M = np.array(Ms)           # shape (n,3)
        ΔAcc = np.array(ΔAcc)      # shape (n,)

        # 2) Spearman ρ_j for each metric j
        rhos = []
        for j in range(M.shape[1]):
            ρ, _ = spearmanr(M[:, j], ΔAcc)
            # handle constant or nan cases
            rhos.append(0.0 if np.isnan(ρ) else float(ρ))
        ρ = np.array(rhos)         # shape (3,)

        # 3) Bayesian-mirror θ update
        self.t += 1
        s = (ρ + 1.0) / 2.0                               # evidence
        θ_bar = (1 - self.lam) * self.theta + self.lam * s
        η_t = self.eta0 / sqrt(self.t)
        g = np.exp(η_t * ρ)                              # mirror tilt
        θ_new = θ_bar * g
        θ_new /= θ_new.sum()                             # re-normalise
        self.theta = θ_new

        # 4) Trust scores T_i = θ·metrics_i
        T = M.dot(self.theta)                            # shape (n,)

        # 5) Dynamic threshold τ(t)
        τ = np.percentile(T, self.percentile)
        trusted = T >= τ

        # 6) Soft-max weights
        temp_t = max(self.temp0 - self.temp_decay * (server_round - 1), self.temp_min)
        w = np.zeros_like(T)
        if trusted.any():
            z = np.exp(T[trusted] / temp_t)
            w[trusted] = z / z.sum()

        # 7) Weighted average of client models
        #    Build list of (weights_i, client_params_i)
        client_arrays = [parameters_to_ndarrays(fit_res.parameters) 
                         for _, fit_res in results]
        # weighted sum over clients
        new_weights = []
        for layer_idx in range(len(client_arrays[0])):
            # sum_i (w_i * layer_i)
            layer_sum = sum(w_i * arr[layer_idx]
                            for w_i, arr in zip(w, client_arrays))
            new_weights.append(layer_sum)

        # 8) Update prev_global_accuracy for next round
        #    Approximate new global accuracy by weighted client accuracies
        self.prev_global_accuracy = float((np.array([fit_res.metrics.get("accuracy", 0.0)
                                                    for _, fit_res in results]) * w).sum())

        # 9) Return new aggregated parameters and empty metrics dict
        return ndarrays_to_parameters(new_weights), {}


In [11]:
# SIMULATION CELL
from flwr.server import ServerApp, ServerAppComponents, ServerConfig
from flwr.simulation import run_simulation
from flwr.client import ClientApp
from flwr.common import Context

def client_fn(context: Context):
    partition_id = int(context.node_config["partition-id"])
    dataset = ToNIoTDataset(partition_id, config.num_clients, config)
    model = SimpleModel(dataset.input_dim, dataset.num_classes)
    return TrustMCClient(model, dataset.train_loader, dataset.test_loader).to_client()

client_app = ClientApp(client_fn)

def server_fn(context):
    strategy = TrustMCStrategy()
    return ServerAppComponents(strategy=strategy, config=ServerConfig(num_rounds=config.num_rounds))

server_app = ServerApp(server_fn)

run_simulation(
    server_app=server_app,
    client_app=client_app,
    num_supernodes=config.num_clients,
    backend_config={"num_cpus": 4}
)



            Check the following `FEATURE UPDATE` warning message for the preferred
            new mechanism to use this feature in Flower.
        
            ------------------------------------------------------------
        

        def server_fn(context: Context):
            server_config = ServerConfig(num_rounds=3)
            strategy = FedAvg()
            return ServerAppComponents(
                strategy=strategy,
                server_config=server_config,
        )

        app = ServerApp(server_fn=server_fn)

            ------------------------------------------------------------
        
INFO :      Starting Flower ServerApp, config: num_rounds=1, no round_timeout
ERROR :     Backend `ray`, is not supported. Use any of [] or add support for a new backend.
INFO :      
ERROR :     Unable to import module `ray`.

    To install the necessary dependencies, install `flwr` with the `simulation` extra:

        pip install -U "flwr[simulation]"
    
ERROR :     Serve

RuntimeError: Exception in ServerApp thread